In [1]:
from data import *
from Solver import *
from redcap import *
import time as t
import pygeoda as pg
import pickle as pkl

In [2]:
global_results = pd.read_pickle('../data/comparison_parameters.pkl')
western_results = pd.read_pickle('../data/western_results.pkl')
#western_results = pd.DataFrame(columns=['n_nodes','states','k','min_counties','z','sc','min_size','heterogeneity','size_regions','regions','n_columns','n_partitions','n_cuts','FiO edges','FuO edges','Sort FuO edges','MST','Tree arch','select regions','column generation'])

In [3]:
redcap_results = pd.read_pickle('../data/redcap_results.pkl')#pd.DataFrame(columns=['n_nodes','states','k','heterogeneity','size_regions','regions','time'])

In [4]:
USA_vote_2004 = pd.read_pickle('../data/USA_vote_2004.pkl')
FullO_file = open("../data/USA_vote_2004_FullO.pkl", "rb")
FullO = pkl.load(FullO_file)
dist = pd.read_csv('../data/USA_vote_2004_dist.csv',index_col=0)
dist.columns = dist.columns.astype(int)
neighbors_file = open('../data/USA_vote_2004_neighbors.pkl','rb')
neighbors = pkl.load(neighbors_file)
edges_file = open('../data/USA_vote_2004_edges.pkl','rb')
edges = pkl.load(edges_file)

In [5]:
western_vote_2004 = pd.read_pickle('../data/western_vote_2004.pkl')
western_FullO_file = open("../data/western_vote_2004_FullO.pkl", "rb")
western_FullO = pkl.load(western_FullO_file)
western_dist = pd.read_csv('../data/western_vote_2004_dist.csv',index_col=0)
western_dist.columns = western_dist.columns.astype(int)
western_neighbors_file = open('../data/western_vote_2004_neighbors.pkl','rb')
western_neighbors = pkl.load(western_neighbors_file)
western_edges_file = open('../data/western_vote_2004_edges.pkl','rb')
western_edges = pkl.load(western_edges_file)

In [10]:
def evaluate(graph,k,min_counties,limit_depth,z,sc,min_size,n_best,output,n_nodes,states,talk=False):
    t1 = t.time()
    graph.SCT.find_regions(min_counties, limit_depth, z, sc, 0,n_best,min_size=min_size,talk=talk)
    t2 = t.time()
    graph.times['column generation'] = t2-t1
    h, x = graph.select_k_regions(k)
    output.loc[len(output)] = [n_nodes,states,k,min_counties,limit_depth,z,sc,min_size,n_best,h,[len(r) for r in x],x,len(graph.SCT.dict_regions),graph.SCT.n_partitions,graph.SCT.n_cuts,graph.times['FiO edges'],graph.times['FuO edges'],graph.times['Sort FuO edges'],graph.times['SCT'],graph.times['Tree arch'],graph.times['select regions'],graph.times['column generation']]
    regions = graph.SCT.dict_regions
    he = graph.SCT.dict_he
    arch = graph.SCT.arch
    graph.SCT.clear()
    return regions,he,arch

def comparate_param(states,n_regions,min_counties,limit_depth,z,sc,min_size,n_best,output,param,talk=False):
    graph, n_nodes = None, 0
    res = []
    if states =='western':
        n_nodes = len(western_vote_2004)
        graph = Graph(western_vote_2004.set_index('county_fips',drop=True),['bush_votes_perc'],full_order=(western_FullO,western_dist))
        graph.SCT = Tree(graph.vertex,western_neighbors,western_edges,0,None)
        graph.times['FuO edges'],graph.times['Sort FuO edges'],graph.times['SCT'],graph.times['Tree arch'] = 0.0, 0.0, 0.0, 0.0
    elif states == 'all':
        n_nodes = len(USA_vote_2004)
        graph = Graph(USA_vote_2004.set_index('county_fips',drop=True),['bush_votes_perc'],full_order=(FullO,dist))
        graph.SCT = Tree(graph.vertex,neighbors,edges,0,None)
        graph.times['FuO edges'],graph.times['Sort FuO edges'],graph.times['SCT'],graph.times['Tree arch'] = 0.0, 0.0, 0.0, 0.0
    else:
        data = percentage_bush_2004(states)
        n_nodes = len(data)
        graph = Graph(data.set_index('county_fips',drop=True),['bush_votes_perc'])
        SCT = graph.find_SCT_full_order_CL()
    if param == 'k':
        for k in n_regions:
            res.append(evaluate(graph,k,min_counties,limit_depth,z,sc,min_size,n_best,output,n_nodes,states,talk=talk))
            print('k = '+str(k)+ ' done')
    elif param == 'min_part':
        for min_c in min_counties:
            res.append(evaluate(graph,n_regions,min_c,limit_depth,z,sc,min_size,n_best,output,n_nodes,states,talk=talk))
            print('min_part = '+str(min_c)+ ' done')
    elif param == 'w':
        for size_z in z:
            res.append(evaluate(graph,n_regions,min_counties,limit_depth,size_z,sc,min_size,n_best,output,n_nodes,states,talk=talk))
            print('w = '+str(size_z)+ ' done')
    elif param == 'sc':
        for sc_val in sc:
            res.append(evaluate(graph,n_regions,min_counties,limit_depth,z,sc_val,min_size,n_best,output,n_nodes,states,talk=talk))
            print('sc = '+str(sc_val)+ ' done')
    elif param == 'min_size':
        for min_s in min_size:
            res.append(evaluate(graph,n_regions,min_counties,limit_depth,z,sc,min_s,n_best,output,n_nodes,states,talk=talk))
            print('min_size = '+str(min_s)+ ' done')
    elif param == 'max_depth':
        for depth in limit_depth:
            res.append(evaluate(graph,n_regions,min_counties,depth,z,sc,min_size,n_best,output,n_nodes,states,talk=talk))
            print('max_depth = '+str(depth)+ ' done')
    elif param == 'n_best':
        for n in n_best:
            res.append(evaluate(graph,n_regions,min_counties,limit_depth,z,sc,min_size,n,output,n_nodes,states,talk=talk))
            print('n_best = '+str(n)+ ' done')
    return res
        
    

In [52]:
def evaluate_redcap(states,k,output,method="fullorder-completelinkage"):
    data = percentage_bush_2004(states)
    n_nodes = len(data)
    h, regions, t = redcap(data,k,talk=True)
    output.loc[len(output)] = [n_nodes,states,method,k,h,[len(regions[x]) for x in regions.keys()],regions,t]


# Test with California State

## k = 5

In [8]:
comparate_param(['CALIFORNIA'],5,[1,3,5,10],1,10,global_results,2)

C:\Users\ngolenvaux\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


min_counties = 1 done
min_counties = 3 done
min_counties = 5 done
min_counties = 10 done


In [10]:
comparate_param(['CALIFORNIA'],5,5,1,[5,10,20,40],global_results,4)

sc = 5 done
sc = 10 done
sc = 20 done
sc = 40 done


In [13]:
comparate_param(['CALIFORNIA'],5,[5,10,20],2,10,global_results,2)

min_counties = 5 done
min_counties = 10 done
min_counties = 20 done


In [16]:
comparate_param(['CALIFORNIA'],5,10,2,[5,10,20,40],global_results,4)

sc = 5 done
sc = 10 done
sc = 20 done
sc = 40 done


In [25]:
comparate_param(['CALIFORNIA'],5,10,3,[10],global_results,4)

sc = 10 done


In [12]:
evaluate_redcap(['CALIFORNIA'],5,redcap_results)

start redcap
end redcap 0.008975744247436523
redcap total heterogeneity :  0.22954071285603542


C:\Users\ngolenvaux\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


## k = 10

In [ ]:
comparate_param(['CALIFORNIA'],10,[3,5,10],1,10,global_results,2)

In [20]:
comparate_param(['CALIFORNIA'],10,5,1,[3,5,10,20],global_results,4)

sc = 3 done
sc = 5 done
sc = 10 done
sc = 20 done


In [22]:
comparate_param(['CALIFORNIA'],10,[3,5,10],2,10,global_results,2)

min_counties = 3 done
min_counties = 5 done
min_counties = 10 done


In [23]:
comparate_param(['CALIFORNIA'],10,5,2,[3,5,10,20],global_results,4)

sc = 3 done
sc = 5 done
sc = 10 done
sc = 20 done


In [13]:
evaluate_redcap(['CALIFORNIA'],10,redcap_results)

start redcap
end redcap 0.007010698318481445
redcap total heterogeneity :  0.12115486320241955


# Test with 5 westerns states

In [28]:
print('start data preparation')
t1 = t.time()
USA_vote_2004 = percentage_bush_2004(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'])
t2 = t.time()
print('end data preparation '+ str(t2-t1))
print('number of nodes: '+ str(len(USA_vote_2004)))

start data preparation
end data preparation 1.1240785121917725
number of nodes: 193


## k = 10

In [36]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,[10,20,40],1,10,global_results,2)

min_counties = 10 done
min_counties = 20 done
min_counties = 40 done


In [38]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,20,1,[5,10,20],global_results,4)

sc = 5 done
sc = 10 done
sc = 20 done


In [40]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,[20,40],2,10,global_results,2)

min_counties = 20 done
min_counties = 40 done


In [43]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,40,2,[5,20,40],global_results,4)

sc = 5 done
sc = 20 done
sc = 40 done


In [15]:
evaluate_redcap(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],10,redcap_results)

start redcap
end redcap 0.02501845359802246
redcap total heterogeneity :  0.6183821682399695


## k=5

In [46]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,[20,40,80],1,10,global_results,2)

min_counties = 20 done
min_counties = 40 done
min_counties = 80 done


In [48]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,40,1,[5,10,20],global_results,4)

sc = 5 done
sc = 10 done
sc = 20 done


In [49]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,[40,80],2,10,global_results,2)

min_counties = 40 done
min_counties = 80 done


In [51]:
comparate_param(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,80,2,[5,20,40],global_results,4)

sc = 5 done
sc = 20 done
sc = 40 done


In [16]:
evaluate_redcap(['CALIFORNIA','WASHINGTON','OREGON','IDAHO','NEVADA'],5,redcap_results)

start redcap
end redcap 0.02469635009765625
redcap total heterogeneity :  1.3064570319460636


# Test with 11 western States

## k=10

In [18]:
arch = comparate_param('western',10,0.05,8,[(1.0,2),(0.2,1)],[(1.0,100),(0.4,50),(0.2,30)],[(1.0,0.1),(0.3,0.1),(0.2,0.0)],[2,5],western_results,'n_best',talk=True)


Node 0 of size 409 and depth = 0
time to find 2 best cuts with sc = 100 : 0.6648976802825928
time to create 2 new trees : 0.004780292510986328
children of node 0 are : [1, 2, 3, 4]

Node 1 of size 276 and depth = 1
time to find 2 best cuts with sc = 100 : 0.2921028137207031
time to create 2 new trees : 0.0026695728302001953
children of node 1 are : [5, 6, 7, 8]

Node 5 of size 183 and depth = 2

Node 6 of size 93 and depth = 2

Node 7 of size 90 and depth = 2

Node 8 of size 186 and depth = 2

Node 2 of size 133 and depth = 1
time to find 2 best cuts with sc = 50 : 0.1244349479675293
time to create 2 new trees : 0.0009999275207519531
children of node 2 are : [715, 716, 717, 718]

Node 715 of size 98 and depth = 2

Node 716 of size 35 and depth = 2

Node 717 of size 36 and depth = 2

Node 718 of size 97 and depth = 2

Node 3 of size 135 and depth = 1
time to find 2 best cuts with sc = 50 : 0.12902283668518066
time to create 2 new trees : 6.413459777832031e-05
children of node 3 are : [

In [25]:
arch = comparate_param('western',10,0.05,[4,5],[(1.0,3),(0.2,1)],[(1.0,100),(0.4,50),(0.2,30)],[(1.0,0.1),(0.3,0.1),(0.2,0.0)],5,western_results,'max_depth',talk=True)


Node 0 of size 409 and depth = 0
time to find 3 best cuts with sc = 100 : 0.674335241317749
time to create 3 new trees : 0.007019519805908203
children of node 0 are : [1, 2, 3, 4, 5, 6]

Node 1 of size 276 and depth = 1
time to find 3 best cuts with sc = 100 : 0.29337382316589355
time to create 3 new trees : 0.003003358840942383
children of node 1 are : [7, 8, 9, 10, 11, 12]

Node 7 of size 183 and depth = 2

Node 8 of size 93 and depth = 2

Node 9 of size 173 and depth = 2

Node 10 of size 103 and depth = 2

Node 11 of size 90 and depth = 2

Node 12 of size 186 and depth = 2

Node 2 of size 133 and depth = 1
time to find 3 best cuts with sc = 50 : 0.12840771675109863
time to create 3 new trees : 0.0015711784362792969
children of node 2 are : [419, 420, 421, 422, 423, 424]

Node 419 of size 98 and depth = 2

Node 420 of size 35 and depth = 2

Node 421 of size 3 and depth = 2

Node 422 of size 130 and depth = 2

Node 423 of size 120 and depth = 2

Node 424 of size 13 and depth = 2

Nod

In [26]:
western_results

,n_nodes,states,k,min_counties,limit_depth,z,sc,min_size,n_best,heterogeneity,...,n_columns,n_partitions,n_cuts,FiO edges,FuO edges,Sort FuO edges,MST,Tree arch,select regions,column generation
0,409,western,10,0.05,8.0,"[(1.0, 2), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",2.0,2.286126,...,1873,811,936,0.218384,0.0,0.0,0.0,0.0,3.496062,73.776776
1,409,western,10,0.05,8.0,"[(1.0, 2), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",5.0,2.262209,...,2213,889,1106,0.218384,0.0,0.0,0.0,0.0,4.155568,84.076801
2,409,western,10,0.05,8.0,"[(1.0, 2), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",10.0,2.262209,...,2175,920,1087,0.199247,0.0,0.0,0.0,0.0,4.357796,87.366841
3,409,western,10,0.05,8.0,"[(1.0, 2), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",20.0,2.289468,...,2299,963,1149,0.199247,0.0,0.0,0.0,0.0,4.326592,100.068100
4,409,western,10,0.05,4.0,"[(1.0, 2), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",5.0,2.517692,...,379,127,189,0.212531,0.0,0.0,0.0,0.0,1.017905,12.646353
5,409,western,10,0.05,5.0,"[(1.0, 2), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",5.0,2.262209,...,681,245,340,0.212531,0.0,0.0,0.0,0.0,1.588780,24.621708
6,409,western,10,0.05,6.0,"[(1.0, 2), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",5.0,2.262209,...,1103,421,551,0.212531,0.0,0.0,0.0,0.0,2.763567,42.370447
7,409,western,10,0.05,7.0,"[(1.0, 2), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",5.0,2.262209,...,1637,641,818,0.212531,0.0,0.0,0.0,0.0,3.889375,71.812114
8,409,western,10,0.05,4.0,"[(1.0, 3), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",5.0,2.517692,...,1965,500,982,0.215142,0.0,0.0,0.0,0.0,6.598539,52.875654
9,409,western,10,0.05,5.0,"[(1.0, 3), (0.2, 1)]","[(1.0, 100), (0.4, 50), (0.2, 30)]","[(1.0, 0.1), (0.3, 0.1), (0.2, 0.0)]",5.0,2.262209,...,4567,1279,2283,0.215142,0.0,0.0,0.0,0.0,13.858817,155.966902


In [30]:
western_results.to_pickle('../data/western_results.pkl')

In [29]:
evaluate_redcap(western_states,10,redcap_results)

start redcap
end redcap 0.07655715942382812
redcap total heterogeneity :  2.2832425329715433


# Test with all US States

## k=10

In [45]:
arch = comparate_param('all',10,0.005,[5,6],[(1.0,2),(0.2,1)],[(1.0,100)],[(1.0,0.0)],5,global_results,'max_depth',talk=True)


Node 0 of size 3083 and depth = 0
time to find 2 best cuts with sc = 100 : 29.696672916412354
time to create 2 new trees : 0.25537729263305664
children of node 0 are : [1, 2, 3, 4]

Node 1 of size 1126 and depth = 1
time to find 2 best cuts with sc = 100 : 8.106165170669556
time to create 2 new trees : 0.051993608474731445
children of node 1 are : [5, 6, 7, 8]

Node 5 of size 681 and depth = 2

Node 6 of size 445 and depth = 2

Node 7 of size 309 and depth = 2

Node 8 of size 817 and depth = 2

Node 2 of size 1957 and depth = 1
time to find 2 best cuts with sc = 100 : 15.653839588165283
time to create 2 new trees : 0.14347481727600098
children of node 2 are : [135, 136, 137, 138]

Node 135 of size 1305 and depth = 2

Node 136 of size 652 and depth = 2

Node 137 of size 1339 and depth = 2

Node 138 of size 618 and depth = 2

Node 3 of size 1967 and depth = 1
time to find 2 best cuts with sc = 100 : 15.264518737792969
time to create 2 new trees : 0.11698579788208008
children of node 3 a

In [44]:
Tree.clear(Tree)

In [17]:
evaluate_redcap('all',10,redcap_results)

C:\Users\ngolenvaux\Documents\Doctorat\Regionalization\Regionalization_models\src\data.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2004['bush_votes_perc'] = df_2004['candidatevotes'] / df_2004['totalvotes']


start redcap
end redcap 0.8467926979064941
redcap total heterogeneity :  21.53039473787621


In [53]:
evaluate_redcap('all',10,redcap_results,method="fullorder-averagelinkage")

C:\Users\ngolenvaux\Documents\Doctorat\Regionalization\Regionalization_models\src\data.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2004['bush_votes_perc'] = df_2004['candidatevotes'] / df_2004['totalvotes']


start redcap
end redcap 0.9240856170654297
redcap total heterogeneity :  21.53039473787621


In [49]:
redcap_results.insert(2,'method',["fullorder-completelinkage" for _ in range(len(redcap_results))])

In [46]:
global_results.loc[50:]

,n_nodes,states,k,min_counties,max_depth,z,sc,min_size,n_best,heterogeneity,...,n_columns,n_partitions,n_cuts,FiO edges,FuO edges,Sort FuO edges,MST,Tree arch,select regions,column generation
50,193,"[CALIFORNIA, WASHINGTON, OREGON, IDAHO, NEVADA]",5,80,None,2,20,0.0,None,1.306457,...,41,10,20,0.088343,6.652561,0.022336,0.920932,0.010211,0.130467,1.142926
51,193,"[CALIFORNIA, WASHINGTON, OREGON, IDAHO, NEVADA]",5,80,None,2,40,0.0,None,1.306457,...,41,10,20,0.088343,6.652561,0.022336,0.920932,0.010211,0.123629,1.564219
52,3083,all,10,100,None,1,50,0.0,None,21.530395,...,263,131,131,1.469064,1636.840292,10.358184,116.588271,0.477663,2.858063,571.967105
53,3083,all,10,300,None,1,50,0.0,None,21.738180,...,115,57,57,1.469064,1636.840292,10.358184,116.588271,0.477663,2.266970,542.948879
54,3083,all,10,500,None,1,50,0.0,None,21.738180,...,63,31,31,1.469064,1636.840292,10.358184,116.588271,0.477663,1.759164,498.533915
55,3083,all,10,200,None,1,10,0.0,None,22.914511,...,61,30,30,1.410364,1752.552082,10.153550,116.603161,0.497046,1.081937,291.193209
56,3083,all,10,200,None,1,30,0.0,None,22.636286,...,99,49,49,1.410364,1752.552082,10.153550,116.603161,0.497046,1.380502,291.877214
57,3083,all,10,200,None,1,100,0.0,None,21.530395,...,177,88,88,1.410364,1752.552082,10.153550,116.603161,0.497046,2.924498,719.866165
58,3083,all,10,1000,None,2,10,0.0,None,23.555772,...,408660,102209,204420,1.420640,1617.541746,10.259073,138.109255,0.348486,2943.210090,703.047169
59,3083,all,10,1000,None,2,50,0.0,None,24.669719,...,1533,383,766,1.420351,1688.738044,10.355141,115.788785,0.422962,43.519545,8650.308966


In [55]:
redcap_results

,n_nodes,states,method,k,heterogeneity,size_regions,regions,time
0,58,[CALIFORNIA],fullorder-completelinkage,5,0.229541,"[22, 8, 12, 10, 6]","{1: [6093, 6015, 6049, 6105, 6089, 6035, 6103,...",0.008976
1,58,[CALIFORNIA],fullorder-completelinkage,10,0.121155,"[20, 2, 4, 4, 9, 4, 5, 1, 6, 3]","{1: [6093, 6015, 6105, 6089, 6103, 6063, 6007,...",0.007011
2,193,"[CALIFORNIA, WASHINGTON, OREGON, IDAHO, NEVADA]",fullorder-completelinkage,10,0.618382,"[27, 24, 22, 45, 4, 1, 50, 8, 6, 6]","{3: [53019, 53065, 53047, 53051, 16021, 16017,...",0.025018
3,193,"[CALIFORNIA, WASHINGTON, OREGON, IDAHO, NEVADA]",fullorder-completelinkage,5,1.306457,"[49, 80, 50, 8, 6]","{3: [53019, 53065, 53047, 53051, 16021, 16017,...",0.024696
4,3083,all,fullorder-completelinkage,10,21.530395,"[1162, 71, 445, 212, 88, 398, 55, 359, 282, 11]","{1: [27077, 38079, 38095, 38019, 38067, 27069,...",0.846793
5,409,"[CALIFORNIA, NEVADA, ARIZONA, OREGON, WASHINGT...",fullorder-completelinkage,10,2.283243,"[81, 12, 35, 97, 1, 91, 35, 43, 8, 6]","{3: [53019, 53065, 53047, 53051, 16021, 30053,...",0.076557
6,3083,all,fullorder-averagelinkage,10,21.530395,"[1162, 71, 445, 212, 88, 398, 55, 359, 282, 11]","{1: [27077, 38079, 38095, 38019, 38067, 27069,...",0.924086


In [47]:
global_results.to_pickle('../data/comparison_parameters.pkl')

In [54]:
redcap_results.to_pickle('../data/redcap_results.pkl')

In [27]:
sum(global_results['size_regions'][67])

4376

## Class Hierarchy

In [20]:
from treelib import Node
from treelib import Tree as Treerep
def create_hierarchy(regions,he,arch):
    hier = {}
    def create_node(index,parent,depth):
        kids = []
        if index in arch:
            kids = [create_node(i,index,depth+1) for i in arch[index]]
        node = Hierarchy(index,regions[index],he[index],kids,parent,depth)
        hier[index] = node
        return node
    root = create_node(0,None,0)
    return hier, root

In [21]:
class Hierarchy:
    def __init__(self, index, regions, he, kids = [], parent = None, depth = 0):
        self.index = index
        self.regions = regions
        self.size = len(regions)
        self.he = he
        self.kids = kids
        self.depth = depth
        self.n = None
        self.maxd = None
        
    def subnodes(self):
        if self.n is None:
            s = 0
            for kid in self.kids:
                s += 1 + kid.subnodes()
            self.n = s
        return self.n
    
    def maxdepth(self):
        if self.maxd is None:
            if self.kids == []:
                self.maxd = 0
            else:
                depths = []
                for kid in self.kids:
                    depths.append(kid.maxdepth())
                self.maxd = 1 + max(depths)
        return self.maxd
    
    def show(self,deep):
        tree = Treerep()
        self.iter_show(deep,tree,None)
        return tree.show()
        
    def iter_show(self,deep,tree,parent):
        tree.create_node((self.size,self.subnodes(),self.maxdepth()),self.index,parent=parent)
        if deep > 0:
            for kid in self.kids:
                kid.iter_show(deep-1,tree,self.index)
            

In [22]:
hier, root = create_hierarchy(arch[0][0],arch[0][1],arch[0][2])

In [23]:
root.subnodes()

1286

In [24]:
root.maxdepth()

16

In [25]:
root.show(5)

(3083, 1286, 16)
├── (1125, 48, 4)
│   ├── (440, 6, 3)
│   │   ├── (18, 0, 0)
│   │   └── (422, 4, 2)
│   │       ├── (67, 0, 0)
│   │       └── (355, 2, 1)
│   │           ├── (174, 0, 0)
│   │           └── (181, 0, 0)
│   ├── (444, 6, 3)
│   │   ├── (18, 0, 0)
│   │   └── (426, 4, 2)
│   │       ├── (67, 0, 0)
│   │       └── (359, 2, 1)
│   │           ├── (174, 0, 0)
│   │           └── (185, 0, 0)
│   ├── (681, 16, 3)
│   │   ├── (283, 2, 1)
│   │   │   ├── (136, 0, 0)
│   │   │   └── (147, 0, 0)
│   │   ├── (286, 2, 1)
│   │   │   ├── (136, 0, 0)
│   │   │   └── (150, 0, 0)
│   │   ├── (395, 4, 2)
│   │   │   ├── (127, 0, 0)
│   │   │   └── (268, 2, 1)
│   │   │       ├── (84, 0, 0)
│   │   │       └── (184, 0, 0)
│   │   └── (398, 4, 2)
│   │       ├── (130, 0, 0)
│   │       └── (268, 2, 1)
│   │           ├── (84, 0, 0)
│   │           └── (184, 0, 0)
│   └── (685, 16, 3)
│       ├── (283, 2, 1)
│       │   ├── (136, 0, 0)
│       │   └── (147, 0, 0)
│       ├── (286, 2, 1)
│

## Run only once to save data from the whole country

In [2]:
print('start data preparation')
t1 = t.time()
USA_vote_2004 = percentage_bush_2004()
t2 = t.time()
print('end data preparation '+ str(t2-t1))
print('number of nodes: '+ str(len(USA_vote_2004)))
USA_vote_2004.to_pickle('../data/USA_vote_2004.pkl')

start data preparation


C:\Users\ngolenvaux\Documents\Doctorat\Regionalization\Regionalization_models\src\data.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2004['bush_votes_perc'] = df_2004['candidatevotes'] / df_2004['totalvotes']


end data preparation 42.52929496765137
number of nodes: 3083


In [3]:
graph = Graph(USA_vote_2004.set_index('county_fips',drop=True),['bush_votes_perc'],talk=True)

start first_order
end first_order 1.4849398136138916
start full_order
end full_order 2108.0296964645386
start sort full order
stop sort full order 0.0


In [4]:
FullO_file = open('../data/USA_vote_2004_FullO.pkl','wb')
pkl.dump(graph.FullO_E,FullO_file)
graph.dist.to_csv('../data/USA_vote_2004_dist.csv')

In [5]:
MST = graph.find_SCT_full_order_CL()#best_k_regions(k,min_regions,z_min,z_max,n_cut,sc)

start MST
stop MST 132.42447018623352
start Tree arch
stop Tree arch 0.4080657958984375


In [6]:
neighbors_file = open('../data/USA_vote_2004_neighbors.pkl','wb')
pkl.dump(graph.SCT.neighbors,neighbors_file)
edges_file = open('../data/USA_vote_2004_edges.pkl','wb')
pkl.dump(graph.SCT.edges,edges_file)

## Run only one for western states

In [28]:
western_states = ['CALIFORNIA','NEVADA','ARIZONA','OREGON','WASHINGTON','NEW MEXICO','MONTANA','IDAHO','WYOMING','UTAH','COLORADO']


In [15]:
print('start data preparation')
t1 = t.time()
western_vote_2004 = percentage_bush_2004(western_states)
t2 = t.time()
print('end data preparation '+ str(t2-t1))
print('number of nodes: '+ str(len(western_vote_2004)))
western_vote_2004.to_pickle('../data/western_vote_2004.pkl')

start data preparation
end data preparation 6.446820020675659
number of nodes: 409


In [13]:
graph = Graph(western_vote_2004.set_index('county_fips',drop=True),['bush_votes_perc'],talk=True)

start first_order
end first_order 0.4517099857330322
start full_order
end full_order 87.65704536437988
start sort full order
stop sort full order 0.21195483207702637


In [14]:
FullO_file = open('../data/western_vote_2004_FullO.pkl','wb')
pkl.dump(graph.FullO_E,FullO_file)
graph.dist.to_csv('../data/western_vote_2004_dist.csv')

In [15]:
MST = graph.find_SCT_full_order_CL()#best_k_regions(k,min_regions,z_min,z_max,n_cut,sc)

start MST
stop MST 11.142862796783447
start Tree arch
stop Tree arch 0.055013179779052734


In [16]:
neighbors_file = open('../data/western_vote_2004_neighbors.pkl','wb')
pkl.dump(graph.SCT.neighbors,neighbors_file)
edges_file = open('../data/western_vote_2004_edges.pkl','wb')
pkl.dump(graph.SCT.edges,edges_file)

## Other

In [30]:
USA_vote_2004 = pd.read_pickle('../data/USA_vote_2004.pkl')
FullO_file = open("../data/USA_vote_2004_FullO.pkl", "rb")
FullO = pkl.load(FullO_file)
dist = pd.read_csv('../data/USA_vote_2004_dist.csv',index_col=0)
dist.columns = dist.columns.astype(int)
neighbors_file = open('../data/USA_vote_2004_neighbors.pkl','rb')
neighbors = pkl.load(neighbors_file)
edges_file = open('../data/USA_vote_2004_edges.pkl','rb')
edges = pkl.load(edges_file)

In [31]:
graph = Graph(USA_vote_2004.set_index('county_fips',drop=True),['bush_votes_perc'],full_order=(FullO,dist))

start first_order
end first_order 2.020354986190796


In [33]:
graph.MST = Tree(graph.vertex,neighbors,edges,0,None)
graph.MST.h = graph.MST.heterogeneity(graph.vertex)

start Tree arch
stop Tree arch 0.11601805686950684


In [4]:
k=10
min_regions= 300
z_min, z_max = 2, 2
n_cut = 1
sc = 5

In [35]:
print('start finding regions')
t1 = t.time()
graph.MST.find_regions(min_regions, z_min, z_max, n_cut, sc)
t2 = t.time()
print('end finding regions ' + str(t2 - t1))

start finding regions
end finding regions 244.71743035316467


In [36]:
res = graph.select_k_regions(k)

start selecting regions
Academic license - for non-commercial use only - expires 2023-01-28
Using license file C:\Users\ngolenvaux\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3084 rows, 33 columns and 16520 nonzeros
Model fingerprint: 0xc13c0a5d
Variable types: 0 continuous, 33 integer (33 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-03, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 3076 rows and 14 columns
Presolve time: 0.01s
Presolved: 8 rows, 19 columns, 44 nonzeros
Variable types: 0 continuous, 19 integer (19 binary)
Found heuristic solution: objective 25.6809269
Found heuristic solution: objective 25.4645704

Root relaxation: objective 2.533079e+01, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

In [41]:
print('start redcap')
t1 = t.time()
votes_2004 = pg.open(USA_vote_2004)
w = pg.queen_weights(votes_2004)
res_redcap = pg.redcap(k,w,USA_vote_2004[['bush_votes_perc']],"fullorder-completelinkage")
t2 = t.time()
print('end redcap '+ str(t2-t1))
clusters_r = {}
for i, c in enumerate(res_redcap['Clusters']):
    if c in clusters_r:
        clusters_r[c] += [USA_vote_2004.loc[i,'FIPS']]
    else:
        clusters_r[c] = [USA_vote_2004.loc[i, 'FIPS']]
h_r = [graph.MST.heterogeneity(clusters_r[i]) for i in clusters_r.keys()]
print('redcap total heterogeneity : ', sum(h_r))

start redcap
end redcap 0.7801389694213867
redcap total heterogeneity :  21.53039473787621


In [52]:
len(neighbors)

3083

In [21]:
dist.columns

Int64Index([27077, 53019, 53065, 53047, 53051, 16021, 30053, 30029, 30035,
            30101,
            ...
            51175, 51083, 51163, 51161, 55083,  4012, 35006, 51199, 51153,
            30031],
           dtype='int64', length=3083)

In [20]:
dist.columns = dist.columns.astype(int)